In [17]:
# Part 1: Data Loading and Basic Preprocessing

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('AirData.csv')

# Display basic info
print("Dataset Info:\n", df.info())
print("\nMissing values:\n", df.isnull().sum())
print("\nSample rows:\n", df.head())

# Encode categorical columns if any
label_encoders = {}
for col in df.select_dtypes(include='object').columns:  #return only the columns where the data type is 'object'
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])    #replaces paticular col with numbers:
    label_encoders[col] = le

# Assume the target variable is named 'Severity' (modify if needed)
target_column = 'Severity'
X = df.drop(columns=[target_column])
y = df[target_column]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   #x-input y-target

print("\nTrain-Test split completed.")
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Severity                 10000 non-null  object 
 1   Safety_Score             10000 non-null  float64
 2   Days_Since_Inspection    10000 non-null  int64  
 3   Total_Safety_Complaints  10000 non-null  int64  
 4   Control_Metric           10000 non-null  float64
 5   Turbulence_In_gforces    10000 non-null  float64
 6   Cabin_Temperature        10000 non-null  float64
 7   Accident_Type_Code       10000 non-null  int64  
 8   Max_Elevation            10000 non-null  float64
 9   Violations               10000 non-null  int64  
 10  Adverse_Weather_Metric   10000 non-null  float64
 11  Accident_ID              10000 non-null  int64  
dtypes: float64(6), int64(5), object(1)
memory usage: 937.6+ KB
Dataset Info:
 None

Missing values:
 Severity                   0
Safet

In [18]:
from sklearn.ensemble import VotingClassifier

voting_model = VotingClassifier(
    estimators=[
        ('mlp', MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=500, early_stopping=True, random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=150, random_state=42)),
        ('xgb', xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
    ],
    voting='soft'
)

voting_model.fit(X_train_scaled, y_train)
y_pred_vote = voting_model.predict(X_test_scaled)

print("\n🔹 Soft Voting (MLP + RF + XGB) Results:")
print("Accuracy:", accuracy_score(y_test, y_pred_vote))
print("Classification Report:\n", classification_report(y_test, y_pred_vote))


C:\Users\Speed Trans\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [16:22:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



🔹 Soft Voting (MLP + RF + XGB) Results:
Accuracy: 0.954
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.93      0.95       597
           1       0.95      0.97      0.96       493
           2       0.94      0.96      0.95       327
           3       0.95      0.96      0.95       583

    accuracy                           0.95      2000
   macro avg       0.95      0.96      0.95      2000
weighted avg       0.95      0.95      0.95      2000

